In [1]:
import random

import numpy as np
import torch
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm import tqdm
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [2]:
data = np.load(r'D:\heat_wave\atlantic\expand_area1\last\optimized_formula_all_variables_expand_atlantic1_surface.npz')
print(data.files)  # ['sst_obser', 'slfh', 'sshf', 'ssr', 'str', 'mld', 'analysis_temp', 'u', 'v', 'T_d', 'u_d', 'v_d', 'xx', 'yy', 'dT_dt', 'dT_dx', 'dT_dy']

sst_obser = data['sst_obser'][:, :80, :79]  # (9861, 41, 201)  #
print(sst_obser.shape)  # (9861, 41, 201)
slfh = data['slfh'][:, :80, :79]  # (9861, 41, 201)
sshf = data['sshf'][:, :80, :79]  # (9861, 41, 201)
ssr = data['ssr'][:, :80, :79]  # (9861, 41, 201)   # 可惜这个是 0
str = data['str'][:, :80, :79]  # (9861, 41, 201)
mld = data['mld'][:, :80, :79]  # (9861, 41, 201)
analysis_temp = data['analysis_temp'][:, :80, :79]  # (9861, 41, 201)
u = data['u'][:, :80, :79]  # (9861, 41, 201)
v = data['v'][:, :80, :79]  # (9861, 41, 201)
T_d = data['T_d'][:, :80, :79]  # (9861, 41, 201)
u_d = data['u_d'][:, :80, :79]  # (9861, 41, 201)
v_d = data['v_d'][:, :80, :79]  # (9861, 41, 201)
xx = data['xx'][:, :80, :79]  # (9861, 41, 201)
yy = data['yy'][:, :80, :79]  # (9861, 41, 201)
print(np.min(T_d))
print(np.max(T_d))
# print(xx)  # -5 -4.75 ... 4.75
# print(yy)  # 190  190.25 ... 239.75

dT_dt = data['dT_dt'][:, :80, :79]
dT_dx = data['dT_dx'][:, :80, :79]
dT_dy = data['dT_dy'][:, :80, :79]


['slfh', 'sshf', 'ssr', 'str', 'sst_obser', 'T_d', 'u_d', 'v_d', 'mld', 'analysis_temp', 'u', 'v', 'dT_dt', 'xx', 'yy', 'dT_dx', 'dT_dy']
(9861, 80, 79)
22.492352
36.701355


In [3]:
Q_net = (slfh + sshf + ssr + str)
Q_net.shape
Q_net = Q_net / (1025 * 4000 * mld)

In [4]:
ZAdv = u * dT_dx
ZAdv.shape

(9861, 80, 79)

In [7]:
MAdv = v * dT_dy
MAdv.shape

(9861, 80, 79)

In [8]:
VAdv = v_d * ((sst_obser -T_d ) / mld)
VAdv.shape

(9861, 80, 79)

In [9]:
R_origin = dT_dt - (Q_net[:9860,:,:] + ZAdv[:9860,:,:] + MAdv[:9860,:,:] + VAdv[:9860,:,:])
R_origin.shape

(9860, 80, 79)

In [10]:
"""我们方法修正项"""

'我们方法修正项'

In [11]:
data1 = np.load(r'D:\heat_wave\atlantic\expand_area1\last\Dt_with_R_e1.npz')
data2 = np.load(r'D:\heat_wave\atlantic\expand_area1\last\Dt_with_R_e2.npz')
data3 = np.load(r'D:\heat_wave\atlantic\expand_area1\last\Dt_with_R_e3.npz')
data4 = np.load(r'D:\heat_wave\atlantic\expand_area1\last\Dt_with_R_e4.npz')
data5 = np.load(r'D:\heat_wave\atlantic\expand_area1\last\Dt_with_R_e5.npz')
data1.files

['dt_R']

In [12]:
dt_R1 = data1['dt_R'][:].reshape(9860,80,79)
dt_R2 = data2['dt_R'][:].reshape(9860,80,79)
dt_R3 = data3['dt_R'][:].reshape(9860,80,79)
dt_R4 = data4['dt_R'][:].reshape(9860,80,79)
dt_R5 = data5['dt_R'][:].reshape(9860,80,79)
dt_R1.shape

(9860, 80, 79)

In [13]:
"修正后的R"
R_recur1 = dt_R1 - dT_dt
R_recur2 = dt_R2 - dT_dt
R_recur3 = dt_R3 - dT_dt
R_recur4 = dt_R4 - dT_dt
R_recur5 = dt_R5 - dT_dt
R_recur1

array([[[ 0.03586979,  0.03347259,  0.05957322, ...,  0.18646735,
          0.19701599,  0.20121458],
        [ 0.05055882,  0.04700889,  0.06782705, ...,  0.19567284,
          0.20308302,  0.20796981],
        [ 0.05712174,  0.06273837,  0.07854559, ...,  0.19160873,
          0.20715764,  0.20956194],
        ...,
        [ 0.15932229,  0.11547898,  0.10034126, ...,  0.16585273,
          0.16188954,  0.13897759],
        [ 0.15250617,  0.14199658,  0.14207235, ...,  0.21118517,
          0.19232756,  0.13984802],
        [ 0.16378003,  0.16585186,  0.16543409, ...,  0.2001392 ,
          0.17457803,  0.13594787]],

       [[ 0.03044586,  0.02774056,  0.04302625, ...,  0.0486625 ,
          0.06338446,  0.06865058],
        [ 0.03752837,  0.03035252,  0.04698587, ...,  0.07801766,
          0.10162099,  0.09557401],
        [ 0.04620308,  0.03525616,  0.04793969, ...,  0.09887834,
          0.12548174,  0.12663248],
        ...,
        [ 0.12780592,  0.09289791,  0.08991582, ...,  

In [14]:
R_recur = ((R_recur1 + R_recur2 + R_recur3 + R_recur4 + R_recur5) / 5)
R_recur

array([[[ 0.02390832,  0.02725428,  0.05739463, ...,  0.18423201,
          0.20143357,  0.212904  ],
        [ 0.03836343,  0.04028901,  0.06597659, ...,  0.19318132,
          0.20801635,  0.21969919],
        [ 0.04502613,  0.05579453,  0.07682253, ...,  0.18921772,
          0.21201761,  0.22122557],
        ...,
        [ 0.14401527,  0.11933136,  0.09818447, ...,  0.16285832,
          0.16673253,  0.15164636],
        [ 0.14027078,  0.13991936,  0.14181487, ...,  0.20777342,
          0.19717088,  0.15250286],
        [ 0.15274742,  0.16048196,  0.16502085, ...,  0.1965462 ,
          0.18078193,  0.14906101]],

       [[ 0.01874388,  0.02132713,  0.04093076, ...,  0.04672079,
          0.06744719,  0.07988962],
        [ 0.02589727,  0.02345898,  0.0454662 , ...,  0.07614927,
          0.1055768 ,  0.10671985],
        [ 0.03458868,  0.02821356,  0.04669535, ...,  0.09688549,
          0.12964737,  0.1379139 ],
        ...,
        [ 0.11189172,  0.09634877,  0.08805746, ...,  

In [15]:
# 首先 我们先画 MAM季节的贡献图   MAM： 3  4  5 月  即  03/01 - 05/31  当然，会做一个区域平均
# 同时，我们也会对每一年的JJA做一个平均  ---》1993 - 2019年

In [16]:
#  计算索引
import pandas as pd
import datetime

#创建时间范围
start_date = datetime.datetime(1993,1,1)
end_date = datetime.datetime(2019,12,31)
dates = pd.date_range(start = start_date, end = end_date, freq = 'D')

index = dates.get_loc(datetime.datetime(2019,5,31)) 
index

9646

In [17]:
9555  9646

SyntaxError: invalid syntax (Temp/ipykernel_22852/1973053253.py, line 1)

In [18]:
# MAM   93
Q_net_MAM_93 = Q_net[59:151,:,:]
ZAdv_MAM_93 = ZAdv[59:151,:,:]
MAdv_MAM_93 = MAdv[59:151,:,:]
VAdv_MAM_93 = VAdv[59:151,:,:]
R_origin_MAM_93 = R_origin[59:151,:,:]
dT_dt_MAM_93 = dT_dt[59:151,:,:]
R_recur_MAM_93 = R_recur[59:151,:,:]

In [19]:
# MAM   94
Q_net_MAM_94 = Q_net[424:516,:,:]
ZAdv_MAM_94 = ZAdv[424:516,:,:]
MAdv_MAM_94 = MAdv[424:516,:,:]
VAdv_MAM_94 = VAdv[424:516,:,:]
R_origin_MAM_94 = R_origin[424:516,:,:]
dT_dt_MAM_94 = dT_dt[424:516,:,:]
R_recur_MAM_94 = R_recur[424:516,:,:]

In [20]:
# MAM   95
Q_net_MAM_95 = Q_net[789:881,:,:]
ZAdv_MAM_95 = ZAdv[789:881,:,:]
MAdv_MAM_95 = MAdv[789:881,:,:]
VAdv_MAM_95 = VAdv[789:881,:,:]
R_origin_MAM_95 = R_origin[789:881,:,:]
dT_dt_MAM_95 = dT_dt[789:881,:,:]
R_recur_MAM_95 = R_recur[789:881,:,:]

In [21]:
# MAM   96
Q_net_MAM_96 = Q_net[1155:1247,:,:]
ZAdv_MAM_96 = ZAdv[1155:1247,:,:]
MAdv_MAM_96 = MAdv[1155:1247,:,:]
VAdv_MAM_96 = VAdv[1155:1247,:,:]
R_origin_MAM_96 = R_origin[1155:1247,:,:]
dT_dt_MAM_96 = dT_dt[1155:1247,:,:]
R_recur_MAM_96 = R_recur[1155:1247,:,:]

In [22]:
# MAM   97
Q_net_MAM_97 = Q_net[1520:1611,:,:]
ZAdv_MAM_97 = ZAdv[1520:1611,:,:]
MAdv_MAM_97 = MAdv[1520:1611,:,:]
VAdv_MAM_97 = VAdv[1520:1611,:,:]
R_origin_MAM_97 = R_origin[1520:1611,:,:]
dT_dt_MAM_97 = dT_dt[1520:1611,:,:]
R_recur_MAM_97 = R_recur[1520:1611,:,:]

In [23]:
# MAM   98
Q_net_MAM_98 = Q_net[1885:1977,:,:]
ZAdv_MAM_98 = ZAdv[1885:1977,:,:]
MAdv_MAM_98 = MAdv[1885:1977,:,:]
VAdv_MAM_98 = VAdv[1885:1977,:,:]
R_origin_MAM_98 = R_origin[1885:1977,:,:]
dT_dt_MAM_98 = dT_dt[1885:1977,:,:]
R_recur_MAM_98 = R_recur[1885:1977,:,:]

In [24]:
# MAM   99
Q_net_MAM_99 = Q_net[2250:2342,:,:]
ZAdv_MAM_99 = ZAdv[2250:2342,:,:]
MAdv_MAM_99 = MAdv[2250:2342,:,:]
VAdv_MAM_99 = VAdv[2250:2342,:,:]
R_origin_MAM_99 = R_origin[2250:2342,:,:]
dT_dt_MAM_99 = dT_dt[2250:2342,:,:]
R_recur_MAM_99 = R_recur[2250:2342,:,:]

In [25]:
# MAM   00
Q_net_MAM_00 = Q_net[2616:2708,:,:]
ZAdv_MAM_00 = ZAdv[2616:2708,:,:]
MAdv_MAM_00 = MAdv[2616:2708,:,:]
VAdv_MAM_00 = VAdv[2616:2708,:,:]
R_origin_MAM_00 = R_origin[2616:2708,:,:]
dT_dt_MAM_00 = dT_dt[2616:2708,:,:]
R_recur_MAM_00 = R_recur[2616:2708,:,:]

In [26]:
# MAM   01
Q_net_MAM_01 = Q_net[2981:3073,:,:]
ZAdv_MAM_01 = ZAdv[2981:3073,:,:]
MAdv_MAM_01 = MAdv[2981:3073,:,:]
VAdv_MAM_01 = VAdv[2981:3073,:,:]
R_origin_MAM_01 = R_origin[2981:3073,:,:]
dT_dt_MAM_01 = dT_dt[2981:3073,:,:]
R_recur_MAM_01 = R_recur[2981:3073,:,:]

In [27]:
# MAM   02
Q_net_MAM_02 = Q_net[3346:3438,:,:]
ZAdv_MAM_02 = ZAdv[3346:3438,:,:]
MAdv_MAM_02 = MAdv[3346:3438,:,:]
VAdv_MAM_02 = VAdv[3346:3438,:,:]
R_origin_MAM_02 = R_origin[3346:3438,:,:]
dT_dt_MAM_02 = dT_dt[3346:3438,:,:]
R_recur_MAM_02 = R_recur[3346:3438,:,:]

In [28]:
# MAM   03
Q_net_MAM_03 = Q_net[3711:3803,:,:]
ZAdv_MAM_03 = ZAdv[3711:3803,:,:]
MAdv_MAM_03 = MAdv[3711:3803,:,:]
VAdv_MAM_03 = VAdv[3711:3803,:,:]
R_origin_MAM_03 = R_origin[3711:3803,:,:]
dT_dt_MAM_03 = dT_dt[3711:3803,:,:]
R_recur_MAM_03 = R_recur[3711:3803,:,:]

In [29]:
# MAM   04
Q_net_MAM_04 = Q_net[4077:4169,:,:]
ZAdv_MAM_04 = ZAdv[4077:4169,:,:]
MAdv_MAM_04 = MAdv[4077:4169,:,:]
VAdv_MAM_04 = VAdv[4077:4169,:,:]
R_origin_MAM_04 = R_origin[4077:4169,:,:]
dT_dt_MAM_04 = dT_dt[4077:4169,:,:]
R_recur_MAM_04 = R_recur[4077:4169,:,:]

In [30]:
# MAM   05
Q_net_MAM_05 = Q_net[4442:4534,:,:]
ZAdv_MAM_05 = ZAdv[4442:4534,:,:]
MAdv_MAM_05 = MAdv[4442:4534,:,:]
VAdv_MAM_05 = VAdv[4442:4534,:,:]
R_origin_MAM_05 = R_origin[4442:4534,:,:]
dT_dt_MAM_05 = dT_dt[4442:4534,:,:]
R_recur_MAM_05 = R_recur[4442:4534,:,:]

In [31]:
# MAM   06
Q_net_MAM_06 = Q_net[4807:4899,:,:]
ZAdv_MAM_06 = ZAdv[4807:4899,:,:]
MAdv_MAM_06 = MAdv[4807:4899,:,:]
VAdv_MAM_06 = VAdv[4807:4899,:,:]
R_origin_MAM_06 = R_origin[4807:4899,:,:]
dT_dt_MAM_06 = dT_dt[4807:4899,:,:]
R_recur_MAM_06 = R_recur[4807:4899,:,:]

In [32]:
# MAM   07
Q_net_MAM_07 = Q_net[5172:5264,:,:]
ZAdv_MAM_07 = ZAdv[5172:5264,:,:]
MAdv_MAM_07 = MAdv[5172:5264,:,:]
VAdv_MAM_07 = VAdv[5172:5264,:,:]
R_origin_MAM_07 = R_origin[5172:5264,:,:]
dT_dt_MAM_07 = dT_dt[5172:5264,:,:]
R_recur_MAM_07 = R_recur[5172:5264,:,:]

In [33]:
# MAM   08
Q_net_MAM_08 = Q_net[5538:5630,:,:]
ZAdv_MAM_08 = ZAdv[5538:5630,:,:]
MAdv_MAM_08 = MAdv[5538:5630,:,:]
VAdv_MAM_08 = VAdv[5538:5630,:,:]
R_origin_MAM_08 = R_origin[5538:5630,:,:]
dT_dt_MAM_08 = dT_dt[5538:5630,:,:]
R_recur_MAM_08 = R_recur[5538:5630,:,:]

In [34]:
# MAM   09
Q_net_MAM_09 = Q_net[5903:5995,:,:]
ZAdv_MAM_09 = ZAdv[5903:5995,:,:]
MAdv_MAM_09 = MAdv[5903:5995,:,:]
VAdv_MAM_09 = VAdv[5903:5995,:,:]
R_origin_MAM_09 = R_origin[5903:5995,:,:]
dT_dt_MAM_09 = dT_dt[5903:5995,:,:]
R_recur_MAM_09 = R_recur[5903:5995,:,:]

In [35]:
# MAM   10
Q_net_MAM_10 = Q_net[6268:6360,:,:]
ZAdv_MAM_10 = ZAdv[6268:6360,:,:]
MAdv_MAM_10 = MAdv[6268:6360,:,:]
VAdv_MAM_10 = VAdv[6268:6360,:,:]
R_origin_MAM_10 = R_origin[6268:6360,:,:]
dT_dt_MAM_10 = dT_dt[6268:6360,:,:]
R_recur_MAM_10 = R_recur[6268:6360,:,:]

In [36]:
# MAM   11
Q_net_MAM_11 = Q_net[6633:6725,:,:]
ZAdv_MAM_11 = ZAdv[6633:6725,:,:]
MAdv_MAM_11 = MAdv[6633:6725,:,:]
VAdv_MAM_11 = VAdv[6633:6725,:,:]
R_origin_MAM_11 = R_origin[6633:6725,:,:]
dT_dt_MAM_11 = dT_dt[6633:6725,:,:]
R_recur_MAM_11 = R_recur[6633:6725,:,:]

In [37]:
# MAM   12
Q_net_MAM_12 = Q_net[6999:7091,:,:]
ZAdv_MAM_12 = ZAdv[6999:7091,:,:]
MAdv_MAM_12 = MAdv[6999:7091,:,:]
VAdv_MAM_12 = VAdv[6999:7091,:,:]
R_origin_MAM_12 = R_origin[6999:7091,:,:]
dT_dt_MAM_12 = dT_dt[6999:7091,:,:]
R_recur_MAM_12 = R_recur[6999:7091,:,:] 

In [38]:
# MAM   13
Q_net_MAM_13 = Q_net[7364:7455,:,:]
ZAdv_MAM_13 = ZAdv[7364:7455,:,:]
MAdv_MAM_13 = MAdv[7364:7455,:,:]
VAdv_MAM_13 = VAdv[7364:7455,:,:]
R_origin_MAM_13 = R_origin[7364:7455,:,:]
dT_dt_MAM_13 = dT_dt[7364:7455,:,:]
R_recur_MAM_13 = R_recur[7364:7455,:,:]

In [39]:
# MAM   14
Q_net_MAM_14 = Q_net[7729:7821,:,:]
ZAdv_MAM_14 = ZAdv[7729:7821,:,:]
MAdv_MAM_14 = MAdv[7729:7821,:,:]
VAdv_MAM_14 = VAdv[7729:7821,:,:]
R_origin_MAM_14 = R_origin[7729:7821,:,:]
dT_dt_MAM_14 = dT_dt[7729:7821,:,:]
R_recur_MAM_14 = R_recur[7729:7821,:,:]

In [40]:
# MAM   15
Q_net_MAM_15 = Q_net[8094:8185,:,:]
ZAdv_MAM_15 = ZAdv[8094:8185,:,:]
MAdv_MAM_15 = MAdv[8094:8185,:,:]
VAdv_MAM_15 = VAdv[8094:8185,:,:]
R_origin_MAM_15 = R_origin[8094:8185,:,:]
dT_dt_MAM_15 = dT_dt[8094:8185,:,:]
R_recur_MAM_15 = R_recur[8094:8185,:,:] 

In [41]:
# MAM   16
Q_net_MAM_16 = Q_net[8460:8552,:,:]
ZAdv_MAM_16 = ZAdv[8460:8552,:,:]
MAdv_MAM_16 = MAdv[8460:8552,:,:]
VAdv_MAM_16 = VAdv[8460:8552,:,:]
R_origin_MAM_16 = R_origin[8460:8552,:,:]
dT_dt_MAM_16 = dT_dt[8460:8552,:,:]
R_recur_MAM_16 = R_recur[8460:8552,:,:] 

In [42]:
# MAM   17
Q_net_MAM_17 = Q_net[8825:8917,:,:]
ZAdv_MAM_17 = ZAdv[8825:8917,:,:]
MAdv_MAM_17 = MAdv[8825:8917,:,:]
VAdv_MAM_17 = VAdv[8825:8917,:,:]
R_origin_MAM_17 = R_origin[8825:8917,:,:]
dT_dt_MAM_17 = dT_dt[8825:8917,:,:]
R_recur_MAM_17 = R_recur[8825:8917,:,:]  

In [43]:
# MAM   18
Q_net_MAM_18 = Q_net[9190:9282,:,:]
ZAdv_MAM_18 = ZAdv[9190:9282,:,:]
MAdv_MAM_18 = MAdv[9190:9282,:,:]
VAdv_MAM_18 = VAdv[9190:9282,:,:]
R_origin_MAM_18 = R_origin[9190:9282,:,:]
dT_dt_MAM_18 = dT_dt[9190:9282,:,:]
R_recur_MAM_18 = R_recur[9190:9282,:,:] 

In [44]:
# MAM   19
Q_net_MAM_19 = Q_net[9555:9646,:,:]
ZAdv_MAM_19 = ZAdv[9555:9646,:,:]
MAdv_MAM_19 = MAdv[9555:9646,:,:]
VAdv_MAM_19 = VAdv[9555:9646,:,:]
R_origin_MAM_19 = R_origin[9555:9646,:,:]
dT_dt_MAM_19 = dT_dt[9555:9646,:,:]
R_recur_MAM_19 = R_recur[9555:9646,:,:]

In [45]:
Q_net_MAM_93 = np.mean(Q_net_MAM_93, axis = (1,2)).reshape(-1,1)
Q_net_MAM_94 = np.mean(Q_net_MAM_94, axis = (1,2)).reshape(-1,1)
Q_net_MAM_95 = np.mean(Q_net_MAM_95, axis = (1,2)).reshape(-1,1)
Q_net_MAM_96 = np.mean(Q_net_MAM_96, axis = (1,2)).reshape(-1,1)
Q_net_MAM_97 = np.mean(Q_net_MAM_97, axis = (1,2)).reshape(-1,1)
Q_net_MAM_98 = np.mean(Q_net_MAM_98, axis = (1,2)).reshape(-1,1)
Q_net_MAM_99 = np.mean(Q_net_MAM_99, axis = (1,2)).reshape(-1,1)
Q_net_MAM_00 = np.mean(Q_net_MAM_00, axis = (1,2)).reshape(-1,1)
Q_net_MAM_01 = np.mean(Q_net_MAM_01, axis = (1,2)).reshape(-1,1)
Q_net_MAM_02 = np.mean(Q_net_MAM_02, axis = (1,2)).reshape(-1,1)
Q_net_MAM_03 = np.mean(Q_net_MAM_03, axis = (1,2)).reshape(-1,1)
Q_net_MAM_04 = np.mean(Q_net_MAM_04, axis = (1,2)).reshape(-1,1)
Q_net_MAM_05 = np.mean(Q_net_MAM_05, axis = (1,2)).reshape(-1,1)
Q_net_MAM_06 = np.mean(Q_net_MAM_06, axis = (1,2)).reshape(-1,1)
Q_net_MAM_07 = np.mean(Q_net_MAM_07, axis = (1,2)).reshape(-1,1)
Q_net_MAM_08 = np.mean(Q_net_MAM_08, axis = (1,2)).reshape(-1,1)
Q_net_MAM_09 = np.mean(Q_net_MAM_09, axis = (1,2)).reshape(-1,1)
Q_net_MAM_10 = np.mean(Q_net_MAM_10, axis = (1,2)).reshape(-1,1)
Q_net_MAM_11 = np.mean(Q_net_MAM_11, axis = (1,2)).reshape(-1,1)
Q_net_MAM_12 = np.mean(Q_net_MAM_12, axis = (1,2)).reshape(-1,1)
Q_net_MAM_13 = np.mean(Q_net_MAM_13, axis = (1,2)).reshape(-1,1)
Q_net_MAM_14 = np.mean(Q_net_MAM_14, axis = (1,2)).reshape(-1,1)
Q_net_MAM_15 = np.mean(Q_net_MAM_15, axis = (1,2)).reshape(-1,1)
Q_net_MAM_16 = np.mean(Q_net_MAM_16, axis = (1,2)).reshape(-1,1)
Q_net_MAM_17 = np.mean(Q_net_MAM_17, axis = (1,2)).reshape(-1,1)
Q_net_MAM_18 = np.mean(Q_net_MAM_18, axis = (1,2)).reshape(-1,1)
Q_net_MAM_19 = np.mean(Q_net_MAM_19, axis = (1,2)).reshape(-1,1)

In [46]:
Q_net_MAM_93 = np.mean(Q_net_MAM_93).reshape(-1,1)
Q_net_MAM_94 = np.mean(Q_net_MAM_94).reshape(-1,1)
Q_net_MAM_95 = np.mean(Q_net_MAM_95).reshape(-1,1)
Q_net_MAM_96 = np.mean(Q_net_MAM_96).reshape(-1,1)
Q_net_MAM_97 = np.mean(Q_net_MAM_97).reshape(-1,1)
Q_net_MAM_98 = np.mean(Q_net_MAM_98).reshape(-1,1)
Q_net_MAM_99 = np.mean(Q_net_MAM_99).reshape(-1,1)
Q_net_MAM_00 = np.mean(Q_net_MAM_00).reshape(-1,1)
Q_net_MAM_01 = np.mean(Q_net_MAM_01).reshape(-1,1)
Q_net_MAM_02 = np.mean(Q_net_MAM_02).reshape(-1,1)
Q_net_MAM_03 = np.mean(Q_net_MAM_03).reshape(-1,1)
Q_net_MAM_04 = np.mean(Q_net_MAM_04).reshape(-1,1)
Q_net_MAM_05 = np.mean(Q_net_MAM_05).reshape(-1,1)
Q_net_MAM_06 = np.mean(Q_net_MAM_06).reshape(-1,1)
Q_net_MAM_07 = np.mean(Q_net_MAM_07).reshape(-1,1)
Q_net_MAM_08 = np.mean(Q_net_MAM_08).reshape(-1,1)
Q_net_MAM_09 = np.mean(Q_net_MAM_09).reshape(-1,1)
Q_net_MAM_10 = np.mean(Q_net_MAM_10).reshape(-1,1)
Q_net_MAM_11 = np.mean(Q_net_MAM_11).reshape(-1,1)
Q_net_MAM_12 = np.mean(Q_net_MAM_12).reshape(-1,1)
Q_net_MAM_13 = np.mean(Q_net_MAM_13).reshape(-1,1)
Q_net_MAM_14 = np.mean(Q_net_MAM_14).reshape(-1,1)
Q_net_MAM_15 = np.mean(Q_net_MAM_15).reshape(-1,1)
Q_net_MAM_16 = np.mean(Q_net_MAM_16).reshape(-1,1)
Q_net_MAM_17 = np.mean(Q_net_MAM_17).reshape(-1,1)
Q_net_MAM_18 = np.mean(Q_net_MAM_18).reshape(-1,1)
Q_net_MAM_19 = np.mean(Q_net_MAM_19).reshape(-1,1)

In [47]:
ZAdv_MAM_93 = np.mean(ZAdv_MAM_93, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_94 = np.mean(ZAdv_MAM_94, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_95 = np.mean(ZAdv_MAM_95, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_96 = np.mean(ZAdv_MAM_96, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_97 = np.mean(ZAdv_MAM_97, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_98 = np.mean(ZAdv_MAM_98, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_99 = np.mean(ZAdv_MAM_99, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_00 = np.mean(ZAdv_MAM_00, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_01 = np.mean(ZAdv_MAM_01, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_02 = np.mean(ZAdv_MAM_02, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_03 = np.mean(ZAdv_MAM_03, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_04 = np.mean(ZAdv_MAM_04, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_05 = np.mean(ZAdv_MAM_05, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_06 = np.mean(ZAdv_MAM_06, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_07 = np.mean(ZAdv_MAM_07, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_08 = np.mean(ZAdv_MAM_08, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_09 = np.mean(ZAdv_MAM_09, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_10 = np.mean(ZAdv_MAM_10, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_11 = np.mean(ZAdv_MAM_11, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_12 = np.mean(ZAdv_MAM_12, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_13 = np.mean(ZAdv_MAM_13, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_14 = np.mean(ZAdv_MAM_14, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_15 = np.mean(ZAdv_MAM_15, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_16 = np.mean(ZAdv_MAM_16, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_17 = np.mean(ZAdv_MAM_17, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_18 = np.mean(ZAdv_MAM_18, axis = (1,2)).reshape(-1,1)
ZAdv_MAM_19 = np.mean(ZAdv_MAM_19, axis = (1,2)).reshape(-1,1)

In [48]:
ZAdv_MAM_93 = np.mean(ZAdv_MAM_93).reshape(-1,1)
ZAdv_MAM_94 = np.mean(ZAdv_MAM_94).reshape(-1,1)
ZAdv_MAM_95 = np.mean(ZAdv_MAM_95).reshape(-1,1)
ZAdv_MAM_96 = np.mean(ZAdv_MAM_96).reshape(-1,1)
ZAdv_MAM_97 = np.mean(ZAdv_MAM_97).reshape(-1,1)
ZAdv_MAM_98 = np.mean(ZAdv_MAM_98).reshape(-1,1)
ZAdv_MAM_99 = np.mean(ZAdv_MAM_99).reshape(-1,1)
ZAdv_MAM_00 = np.mean(ZAdv_MAM_00).reshape(-1,1)
ZAdv_MAM_01 = np.mean(ZAdv_MAM_01).reshape(-1,1)
ZAdv_MAM_02 = np.mean(ZAdv_MAM_02).reshape(-1,1)
ZAdv_MAM_03 = np.mean(ZAdv_MAM_03).reshape(-1,1)
ZAdv_MAM_04 = np.mean(ZAdv_MAM_04).reshape(-1,1)
ZAdv_MAM_05 = np.mean(ZAdv_MAM_05).reshape(-1,1)
ZAdv_MAM_06 = np.mean(ZAdv_MAM_06).reshape(-1,1)
ZAdv_MAM_07 = np.mean(ZAdv_MAM_07).reshape(-1,1)
ZAdv_MAM_08 = np.mean(ZAdv_MAM_08).reshape(-1,1)
ZAdv_MAM_09 = np.mean(ZAdv_MAM_09).reshape(-1,1)
ZAdv_MAM_10 = np.mean(ZAdv_MAM_10).reshape(-1,1)
ZAdv_MAM_11 = np.mean(ZAdv_MAM_11).reshape(-1,1)
ZAdv_MAM_12 = np.mean(ZAdv_MAM_12).reshape(-1,1)
ZAdv_MAM_13 = np.mean(ZAdv_MAM_13).reshape(-1,1)
ZAdv_MAM_14 = np.mean(ZAdv_MAM_14).reshape(-1,1)
ZAdv_MAM_15 = np.mean(ZAdv_MAM_15).reshape(-1,1)
ZAdv_MAM_16 = np.mean(ZAdv_MAM_16).reshape(-1,1)
ZAdv_MAM_17 = np.mean(ZAdv_MAM_17).reshape(-1,1)
ZAdv_MAM_18 = np.mean(ZAdv_MAM_18).reshape(-1,1)
ZAdv_MAM_19 = np.mean(ZAdv_MAM_19).reshape(-1,1)

In [49]:
MAdv_MAM_93 = np.mean(MAdv_MAM_93, axis = (1,2)).reshape(-1,1)
MAdv_MAM_94 = np.mean(MAdv_MAM_94, axis = (1,2)).reshape(-1,1)
MAdv_MAM_95 = np.mean(MAdv_MAM_95, axis = (1,2)).reshape(-1,1)
MAdv_MAM_96 = np.mean(MAdv_MAM_96, axis = (1,2)).reshape(-1,1)
MAdv_MAM_97 = np.mean(MAdv_MAM_97, axis = (1,2)).reshape(-1,1)
MAdv_MAM_98 = np.mean(MAdv_MAM_98, axis = (1,2)).reshape(-1,1)
MAdv_MAM_99 = np.mean(MAdv_MAM_99, axis = (1,2)).reshape(-1,1)
MAdv_MAM_00 = np.mean(MAdv_MAM_00, axis = (1,2)).reshape(-1,1)
MAdv_MAM_01 = np.mean(MAdv_MAM_01, axis = (1,2)).reshape(-1,1)
MAdv_MAM_02 = np.mean(MAdv_MAM_02, axis = (1,2)).reshape(-1,1)
MAdv_MAM_03 = np.mean(MAdv_MAM_03, axis = (1,2)).reshape(-1,1)
MAdv_MAM_04 = np.mean(MAdv_MAM_04, axis = (1,2)).reshape(-1,1)
MAdv_MAM_05 = np.mean(MAdv_MAM_05, axis = (1,2)).reshape(-1,1)
MAdv_MAM_06 = np.mean(MAdv_MAM_06, axis = (1,2)).reshape(-1,1)
MAdv_MAM_07 = np.mean(MAdv_MAM_07, axis = (1,2)).reshape(-1,1)
MAdv_MAM_08 = np.mean(MAdv_MAM_08, axis = (1,2)).reshape(-1,1)
MAdv_MAM_09 = np.mean(MAdv_MAM_09, axis = (1,2)).reshape(-1,1)
MAdv_MAM_10 = np.mean(MAdv_MAM_10, axis = (1,2)).reshape(-1,1)
MAdv_MAM_11 = np.mean(MAdv_MAM_11, axis = (1,2)).reshape(-1,1)
MAdv_MAM_12 = np.mean(MAdv_MAM_12, axis = (1,2)).reshape(-1,1)
MAdv_MAM_13 = np.mean(MAdv_MAM_13, axis = (1,2)).reshape(-1,1)
MAdv_MAM_14 = np.mean(MAdv_MAM_14, axis = (1,2)).reshape(-1,1)
MAdv_MAM_15 = np.mean(MAdv_MAM_15, axis = (1,2)).reshape(-1,1)
MAdv_MAM_16 = np.mean(MAdv_MAM_16, axis = (1,2)).reshape(-1,1)
MAdv_MAM_17 = np.mean(MAdv_MAM_17, axis = (1,2)).reshape(-1,1)
MAdv_MAM_18 = np.mean(MAdv_MAM_18, axis = (1,2)).reshape(-1,1)
MAdv_MAM_19 = np.mean(MAdv_MAM_19, axis = (1,2)).reshape(-1,1)

In [50]:
MAdv_MAM_93 = np.mean(MAdv_MAM_93).reshape(-1,1)
MAdv_MAM_94 = np.mean(MAdv_MAM_94).reshape(-1,1)
MAdv_MAM_95 = np.mean(MAdv_MAM_95).reshape(-1,1)
MAdv_MAM_96 = np.mean(MAdv_MAM_96).reshape(-1,1)
MAdv_MAM_97 = np.mean(MAdv_MAM_97).reshape(-1,1)
MAdv_MAM_98 = np.mean(MAdv_MAM_98).reshape(-1,1)
MAdv_MAM_99 = np.mean(MAdv_MAM_99).reshape(-1,1)
MAdv_MAM_00 = np.mean(MAdv_MAM_00).reshape(-1,1)
MAdv_MAM_01 = np.mean(MAdv_MAM_01).reshape(-1,1)
MAdv_MAM_02 = np.mean(MAdv_MAM_02).reshape(-1,1)
MAdv_MAM_03 = np.mean(MAdv_MAM_03).reshape(-1,1)
MAdv_MAM_04 = np.mean(MAdv_MAM_04).reshape(-1,1)
MAdv_MAM_05 = np.mean(MAdv_MAM_05).reshape(-1,1)
MAdv_MAM_06 = np.mean(MAdv_MAM_06).reshape(-1,1)
MAdv_MAM_07 = np.mean(MAdv_MAM_07).reshape(-1,1)
MAdv_MAM_08 = np.mean(MAdv_MAM_08).reshape(-1,1)
MAdv_MAM_09 = np.mean(MAdv_MAM_09).reshape(-1,1)
MAdv_MAM_10 = np.mean(MAdv_MAM_10).reshape(-1,1)
MAdv_MAM_11 = np.mean(MAdv_MAM_11).reshape(-1,1)
MAdv_MAM_12 = np.mean(MAdv_MAM_12).reshape(-1,1)
MAdv_MAM_13 = np.mean(MAdv_MAM_13).reshape(-1,1)
MAdv_MAM_14 = np.mean(MAdv_MAM_14).reshape(-1,1)
MAdv_MAM_15 = np.mean(MAdv_MAM_15).reshape(-1,1)
MAdv_MAM_16 = np.mean(MAdv_MAM_16).reshape(-1,1)
MAdv_MAM_17 = np.mean(MAdv_MAM_17).reshape(-1,1)
MAdv_MAM_18 = np.mean(MAdv_MAM_18).reshape(-1,1)
MAdv_MAM_19 = np.mean(MAdv_MAM_19).reshape(-1,1)

In [51]:
VAdv_MAM_93 = np.mean(VAdv_MAM_93, axis = (1,2)).reshape(-1,1)
VAdv_MAM_94 = np.mean(VAdv_MAM_94, axis = (1,2)).reshape(-1,1)
VAdv_MAM_95 = np.mean(VAdv_MAM_95, axis = (1,2)).reshape(-1,1)
VAdv_MAM_96 = np.mean(VAdv_MAM_96, axis = (1,2)).reshape(-1,1)
VAdv_MAM_97 = np.mean(VAdv_MAM_97, axis = (1,2)).reshape(-1,1)
VAdv_MAM_98 = np.mean(VAdv_MAM_98, axis = (1,2)).reshape(-1,1)
VAdv_MAM_99 = np.mean(VAdv_MAM_99, axis = (1,2)).reshape(-1,1)
VAdv_MAM_00 = np.mean(VAdv_MAM_00, axis = (1,2)).reshape(-1,1)
VAdv_MAM_01 = np.mean(VAdv_MAM_01, axis = (1,2)).reshape(-1,1)
VAdv_MAM_02 = np.mean(VAdv_MAM_02, axis = (1,2)).reshape(-1,1)
VAdv_MAM_03 = np.mean(VAdv_MAM_03, axis = (1,2)).reshape(-1,1)
VAdv_MAM_04 = np.mean(VAdv_MAM_04, axis = (1,2)).reshape(-1,1)
VAdv_MAM_05 = np.mean(VAdv_MAM_05, axis = (1,2)).reshape(-1,1)
VAdv_MAM_06 = np.mean(VAdv_MAM_06, axis = (1,2)).reshape(-1,1)
VAdv_MAM_07 = np.mean(VAdv_MAM_07, axis = (1,2)).reshape(-1,1)
VAdv_MAM_08 = np.mean(VAdv_MAM_08, axis = (1,2)).reshape(-1,1)
VAdv_MAM_09 = np.mean(VAdv_MAM_09, axis = (1,2)).reshape(-1,1)
VAdv_MAM_10 = np.mean(VAdv_MAM_10, axis = (1,2)).reshape(-1,1)
VAdv_MAM_11 = np.mean(VAdv_MAM_11, axis = (1,2)).reshape(-1,1)
VAdv_MAM_12 = np.mean(VAdv_MAM_12, axis = (1,2)).reshape(-1,1)
VAdv_MAM_13 = np.mean(VAdv_MAM_13, axis = (1,2)).reshape(-1,1)
VAdv_MAM_14 = np.mean(VAdv_MAM_14, axis = (1,2)).reshape(-1,1)
VAdv_MAM_15 = np.mean(VAdv_MAM_15, axis = (1,2)).reshape(-1,1)
VAdv_MAM_16 = np.mean(VAdv_MAM_16, axis = (1,2)).reshape(-1,1)
VAdv_MAM_17 = np.mean(VAdv_MAM_17, axis = (1,2)).reshape(-1,1)
VAdv_MAM_18 = np.mean(VAdv_MAM_18, axis = (1,2)).reshape(-1,1)
VAdv_MAM_19 = np.mean(VAdv_MAM_19, axis = (1,2)).reshape(-1,1)

In [52]:
VAdv_MAM_93 = np.mean(VAdv_MAM_93).reshape(-1,1)
VAdv_MAM_94 = np.mean(VAdv_MAM_94).reshape(-1,1)
VAdv_MAM_95 = np.mean(VAdv_MAM_95).reshape(-1,1)
VAdv_MAM_96 = np.mean(VAdv_MAM_96).reshape(-1,1)
VAdv_MAM_97 = np.mean(VAdv_MAM_97).reshape(-1,1)
VAdv_MAM_98 = np.mean(VAdv_MAM_98).reshape(-1,1)
VAdv_MAM_99 = np.mean(VAdv_MAM_99).reshape(-1,1)
VAdv_MAM_00 = np.mean(VAdv_MAM_00).reshape(-1,1)
VAdv_MAM_01 = np.mean(VAdv_MAM_01).reshape(-1,1)
VAdv_MAM_02 = np.mean(VAdv_MAM_02).reshape(-1,1)
VAdv_MAM_03 = np.mean(VAdv_MAM_03).reshape(-1,1)
VAdv_MAM_04 = np.mean(VAdv_MAM_04).reshape(-1,1)
VAdv_MAM_05 = np.mean(VAdv_MAM_05).reshape(-1,1)
VAdv_MAM_06 = np.mean(VAdv_MAM_06).reshape(-1,1)
VAdv_MAM_07 = np.mean(VAdv_MAM_07).reshape(-1,1)
VAdv_MAM_08 = np.mean(VAdv_MAM_08).reshape(-1,1)
VAdv_MAM_09 = np.mean(VAdv_MAM_09).reshape(-1,1)
VAdv_MAM_10 = np.mean(VAdv_MAM_10).reshape(-1,1)
VAdv_MAM_11 = np.mean(VAdv_MAM_11).reshape(-1,1)
VAdv_MAM_12 = np.mean(VAdv_MAM_12).reshape(-1,1)
VAdv_MAM_13 = np.mean(VAdv_MAM_13).reshape(-1,1)
VAdv_MAM_14 = np.mean(VAdv_MAM_14).reshape(-1,1)
VAdv_MAM_15 = np.mean(VAdv_MAM_15).reshape(-1,1)
VAdv_MAM_16 = np.mean(VAdv_MAM_16).reshape(-1,1)
VAdv_MAM_17 = np.mean(VAdv_MAM_17).reshape(-1,1)
VAdv_MAM_18 = np.mean(VAdv_MAM_18).reshape(-1,1)
VAdv_MAM_19 = np.mean(VAdv_MAM_19).reshape(-1,1)

In [53]:
R_origin_MAM_93 = np.mean(R_origin_MAM_93, axis = (1,2)).reshape(-1,1)
R_origin_MAM_94 = np.mean(R_origin_MAM_94, axis = (1,2)).reshape(-1,1)
R_origin_MAM_95 = np.mean(R_origin_MAM_95, axis = (1,2)).reshape(-1,1)
R_origin_MAM_96 = np.mean(R_origin_MAM_96, axis = (1,2)).reshape(-1,1)
R_origin_MAM_97 = np.mean(R_origin_MAM_97, axis = (1,2)).reshape(-1,1)
R_origin_MAM_98 = np.mean(R_origin_MAM_98, axis = (1,2)).reshape(-1,1)
R_origin_MAM_99 = np.mean(R_origin_MAM_99, axis = (1,2)).reshape(-1,1)
R_origin_MAM_00 = np.mean(R_origin_MAM_00, axis = (1,2)).reshape(-1,1)
R_origin_MAM_01 = np.mean(R_origin_MAM_01, axis = (1,2)).reshape(-1,1)
R_origin_MAM_02 = np.mean(R_origin_MAM_02, axis = (1,2)).reshape(-1,1)
R_origin_MAM_03 = np.mean(R_origin_MAM_03, axis = (1,2)).reshape(-1,1)
R_origin_MAM_04 = np.mean(R_origin_MAM_04, axis = (1,2)).reshape(-1,1)
R_origin_MAM_05 = np.mean(R_origin_MAM_05, axis = (1,2)).reshape(-1,1)
R_origin_MAM_06 = np.mean(R_origin_MAM_06, axis = (1,2)).reshape(-1,1)
R_origin_MAM_07 = np.mean(R_origin_MAM_07, axis = (1,2)).reshape(-1,1)
R_origin_MAM_08 = np.mean(R_origin_MAM_08, axis = (1,2)).reshape(-1,1)
R_origin_MAM_09 = np.mean(R_origin_MAM_09, axis = (1,2)).reshape(-1,1)
R_origin_MAM_10 = np.mean(R_origin_MAM_10, axis = (1,2)).reshape(-1,1)
R_origin_MAM_11 = np.mean(R_origin_MAM_11, axis = (1,2)).reshape(-1,1)
R_origin_MAM_12 = np.mean(R_origin_MAM_12, axis = (1,2)).reshape(-1,1)
R_origin_MAM_13 = np.mean(R_origin_MAM_13, axis = (1,2)).reshape(-1,1)
R_origin_MAM_14 = np.mean(R_origin_MAM_14, axis = (1,2)).reshape(-1,1)
R_origin_MAM_15 = np.mean(R_origin_MAM_15, axis = (1,2)).reshape(-1,1)
R_origin_MAM_16 = np.mean(R_origin_MAM_16, axis = (1,2)).reshape(-1,1)
R_origin_MAM_17 = np.mean(R_origin_MAM_17, axis = (1,2)).reshape(-1,1)
R_origin_MAM_18 = np.mean(R_origin_MAM_18, axis = (1,2)).reshape(-1,1)
R_origin_MAM_19 = np.mean(R_origin_MAM_19, axis = (1,2)).reshape(-1,1)

In [54]:
R_origin_MAM_93 = np.mean(R_origin_MAM_93).reshape(-1,1)
R_origin_MAM_94 = np.mean(R_origin_MAM_94).reshape(-1,1)
R_origin_MAM_95 = np.mean(R_origin_MAM_95).reshape(-1,1)
R_origin_MAM_96 = np.mean(R_origin_MAM_96).reshape(-1,1)
R_origin_MAM_97 = np.mean(R_origin_MAM_97).reshape(-1,1)
R_origin_MAM_98 = np.mean(R_origin_MAM_98).reshape(-1,1)
R_origin_MAM_99 = np.mean(R_origin_MAM_99).reshape(-1,1)
R_origin_MAM_00 = np.mean(R_origin_MAM_00).reshape(-1,1)
R_origin_MAM_01 = np.mean(R_origin_MAM_01).reshape(-1,1)
R_origin_MAM_02 = np.mean(R_origin_MAM_02).reshape(-1,1)
R_origin_MAM_03 = np.mean(R_origin_MAM_03).reshape(-1,1)
R_origin_MAM_04 = np.mean(R_origin_MAM_04).reshape(-1,1)
R_origin_MAM_05 = np.mean(R_origin_MAM_05).reshape(-1,1)
R_origin_MAM_06 = np.mean(R_origin_MAM_06).reshape(-1,1)
R_origin_MAM_07 = np.mean(R_origin_MAM_07).reshape(-1,1)
R_origin_MAM_08 = np.mean(R_origin_MAM_08).reshape(-1,1)
R_origin_MAM_09 = np.mean(R_origin_MAM_09).reshape(-1,1)
R_origin_MAM_10 = np.mean(R_origin_MAM_10).reshape(-1,1)
R_origin_MAM_11 = np.mean(R_origin_MAM_11).reshape(-1,1)
R_origin_MAM_12 = np.mean(R_origin_MAM_12).reshape(-1,1)
R_origin_MAM_13 = np.mean(R_origin_MAM_13).reshape(-1,1)
R_origin_MAM_14 = np.mean(R_origin_MAM_14).reshape(-1,1)
R_origin_MAM_15 = np.mean(R_origin_MAM_15).reshape(-1,1)
R_origin_MAM_16 = np.mean(R_origin_MAM_16).reshape(-1,1)
R_origin_MAM_17 = np.mean(R_origin_MAM_17).reshape(-1,1)
R_origin_MAM_18 = np.mean(R_origin_MAM_18).reshape(-1,1)
R_origin_MAM_19 = np.mean(R_origin_MAM_19).reshape(-1,1)

In [55]:
dT_dt_MAM_93 = np.mean(dT_dt_MAM_93, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_94 = np.mean(dT_dt_MAM_94, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_95 = np.mean(dT_dt_MAM_95, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_96 = np.mean(dT_dt_MAM_96, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_97 = np.mean(dT_dt_MAM_97, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_98 = np.mean(dT_dt_MAM_98, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_99 = np.mean(dT_dt_MAM_99, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_00 = np.mean(dT_dt_MAM_00, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_01 = np.mean(dT_dt_MAM_01, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_02 = np.mean(dT_dt_MAM_02, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_03 = np.mean(dT_dt_MAM_03, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_04 = np.mean(dT_dt_MAM_04, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_05 = np.mean(dT_dt_MAM_05, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_06 = np.mean(dT_dt_MAM_06, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_07 = np.mean(dT_dt_MAM_07, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_08 = np.mean(dT_dt_MAM_08, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_09 = np.mean(dT_dt_MAM_09, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_10 = np.mean(dT_dt_MAM_10, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_11 = np.mean(dT_dt_MAM_11, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_12 = np.mean(dT_dt_MAM_12, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_13 = np.mean(dT_dt_MAM_13, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_14 = np.mean(dT_dt_MAM_14, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_15 = np.mean(dT_dt_MAM_15, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_16 = np.mean(dT_dt_MAM_16, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_17 = np.mean(dT_dt_MAM_17, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_18 = np.mean(dT_dt_MAM_18, axis = (1,2)).reshape(-1,1)
dT_dt_MAM_19 = np.mean(dT_dt_MAM_19, axis = (1,2)).reshape(-1,1)

In [56]:
dT_dt_MAM_93 = np.mean(dT_dt_MAM_93).reshape(-1,1)
dT_dt_MAM_94 = np.mean(dT_dt_MAM_94).reshape(-1,1)
dT_dt_MAM_95 = np.mean(dT_dt_MAM_95).reshape(-1,1)
dT_dt_MAM_96 = np.mean(dT_dt_MAM_96).reshape(-1,1)
dT_dt_MAM_97 = np.mean(dT_dt_MAM_97).reshape(-1,1)
dT_dt_MAM_98 = np.mean(dT_dt_MAM_98).reshape(-1,1)
dT_dt_MAM_99 = np.mean(dT_dt_MAM_99).reshape(-1,1)
dT_dt_MAM_00 = np.mean(dT_dt_MAM_00).reshape(-1,1)
dT_dt_MAM_01 = np.mean(dT_dt_MAM_01).reshape(-1,1)
dT_dt_MAM_02 = np.mean(dT_dt_MAM_02).reshape(-1,1)
dT_dt_MAM_03 = np.mean(dT_dt_MAM_03).reshape(-1,1)
dT_dt_MAM_04 = np.mean(dT_dt_MAM_04).reshape(-1,1)
dT_dt_MAM_05 = np.mean(dT_dt_MAM_05).reshape(-1,1)
dT_dt_MAM_06 = np.mean(dT_dt_MAM_06).reshape(-1,1)
dT_dt_MAM_07 = np.mean(dT_dt_MAM_07).reshape(-1,1)
dT_dt_MAM_08 = np.mean(dT_dt_MAM_08).reshape(-1,1)
dT_dt_MAM_09 = np.mean(dT_dt_MAM_09).reshape(-1,1)
dT_dt_MAM_10 = np.mean(dT_dt_MAM_10).reshape(-1,1)
dT_dt_MAM_11 = np.mean(dT_dt_MAM_11).reshape(-1,1)
dT_dt_MAM_12 = np.mean(dT_dt_MAM_12).reshape(-1,1)
dT_dt_MAM_13 = np.mean(dT_dt_MAM_13).reshape(-1,1)
dT_dt_MAM_14 = np.mean(dT_dt_MAM_14).reshape(-1,1)
dT_dt_MAM_15 = np.mean(dT_dt_MAM_15).reshape(-1,1)
dT_dt_MAM_16 = np.mean(dT_dt_MAM_16).reshape(-1,1)
dT_dt_MAM_17 = np.mean(dT_dt_MAM_17).reshape(-1,1)
dT_dt_MAM_18 = np.mean(dT_dt_MAM_18).reshape(-1,1)
dT_dt_MAM_19 = np.mean(dT_dt_MAM_19).reshape(-1,1)

In [57]:
R_recur_MAM_93 = np.mean(R_recur_MAM_93, axis = (1,2)).reshape(-1,1)
R_recur_MAM_94 = np.mean(R_recur_MAM_94, axis = (1,2)).reshape(-1,1)
R_recur_MAM_95 = np.mean(R_recur_MAM_95, axis = (1,2)).reshape(-1,1)
R_recur_MAM_96 = np.mean(R_recur_MAM_96, axis = (1,2)).reshape(-1,1)
R_recur_MAM_97 = np.mean(R_recur_MAM_97, axis = (1,2)).reshape(-1,1)
R_recur_MAM_98 = np.mean(R_recur_MAM_98, axis = (1,2)).reshape(-1,1)
R_recur_MAM_99 = np.mean(R_recur_MAM_99, axis = (1,2)).reshape(-1,1)
R_recur_MAM_00 = np.mean(R_recur_MAM_00, axis = (1,2)).reshape(-1,1)
R_recur_MAM_01 = np.mean(R_recur_MAM_01, axis = (1,2)).reshape(-1,1)
R_recur_MAM_02 = np.mean(R_recur_MAM_02, axis = (1,2)).reshape(-1,1)
R_recur_MAM_03 = np.mean(R_recur_MAM_03, axis = (1,2)).reshape(-1,1)
R_recur_MAM_04 = np.mean(R_recur_MAM_04, axis = (1,2)).reshape(-1,1)
R_recur_MAM_05 = np.mean(R_recur_MAM_05, axis = (1,2)).reshape(-1,1)
R_recur_MAM_06 = np.mean(R_recur_MAM_06, axis = (1,2)).reshape(-1,1)
R_recur_MAM_07 = np.mean(R_recur_MAM_07, axis = (1,2)).reshape(-1,1)
R_recur_MAM_08 = np.mean(R_recur_MAM_08, axis = (1,2)).reshape(-1,1)
R_recur_MAM_09 = np.mean(R_recur_MAM_09, axis = (1,2)).reshape(-1,1)
R_recur_MAM_10 = np.mean(R_recur_MAM_10, axis = (1,2)).reshape(-1,1)
R_recur_MAM_11 = np.mean(R_recur_MAM_11, axis = (1,2)).reshape(-1,1)
R_recur_MAM_12 = np.mean(R_recur_MAM_12, axis = (1,2)).reshape(-1,1)
R_recur_MAM_13 = np.mean(R_recur_MAM_13, axis = (1,2)).reshape(-1,1)
R_recur_MAM_14 = np.mean(R_recur_MAM_14, axis = (1,2)).reshape(-1,1)
R_recur_MAM_15 = np.mean(R_recur_MAM_15, axis = (1,2)).reshape(-1,1)
R_recur_MAM_16 = np.mean(R_recur_MAM_16, axis = (1,2)).reshape(-1,1)
R_recur_MAM_17 = np.mean(R_recur_MAM_17, axis = (1,2)).reshape(-1,1)
R_recur_MAM_18 = np.mean(R_recur_MAM_18, axis = (1,2)).reshape(-1,1)
R_recur_MAM_19 = np.mean(R_recur_MAM_19, axis = (1,2)).reshape(-1,1)

In [58]:
R_recur_MAM_93 = np.mean(R_recur_MAM_93).reshape(-1,1)
R_recur_MAM_94 = np.mean(R_recur_MAM_94).reshape(-1,1)
R_recur_MAM_95 = np.mean(R_recur_MAM_95).reshape(-1,1)
R_recur_MAM_96 = np.mean(R_recur_MAM_96).reshape(-1,1)
R_recur_MAM_97 = np.mean(R_recur_MAM_97).reshape(-1,1)
R_recur_MAM_98 = np.mean(R_recur_MAM_98).reshape(-1,1)
R_recur_MAM_99 = np.mean(R_recur_MAM_99).reshape(-1,1)
R_recur_MAM_00 = np.mean(R_recur_MAM_00).reshape(-1,1)
R_recur_MAM_01 = np.mean(R_recur_MAM_01).reshape(-1,1)
R_recur_MAM_02 = np.mean(R_recur_MAM_02).reshape(-1,1)
R_recur_MAM_03 = np.mean(R_recur_MAM_03).reshape(-1,1)
R_recur_MAM_04 = np.mean(R_recur_MAM_04).reshape(-1,1)
R_recur_MAM_05 = np.mean(R_recur_MAM_05).reshape(-1,1)
R_recur_MAM_06 = np.mean(R_recur_MAM_06).reshape(-1,1)
R_recur_MAM_07 = np.mean(R_recur_MAM_07).reshape(-1,1)
R_recur_MAM_08 = np.mean(R_recur_MAM_08).reshape(-1,1)
R_recur_MAM_09 = np.mean(R_recur_MAM_09).reshape(-1,1)
R_recur_MAM_10 = np.mean(R_recur_MAM_10).reshape(-1,1)
R_recur_MAM_11 = np.mean(R_recur_MAM_11).reshape(-1,1)
R_recur_MAM_12 = np.mean(R_recur_MAM_12).reshape(-1,1)
R_recur_MAM_13 = np.mean(R_recur_MAM_13).reshape(-1,1)
R_recur_MAM_14 = np.mean(R_recur_MAM_14).reshape(-1,1)
R_recur_MAM_15 = np.mean(R_recur_MAM_15).reshape(-1,1)
R_recur_MAM_16 = np.mean(R_recur_MAM_16).reshape(-1,1)
R_recur_MAM_17 = np.mean(R_recur_MAM_17).reshape(-1,1)
R_recur_MAM_18 = np.mean(R_recur_MAM_18).reshape(-1,1)
R_recur_MAM_19 = np.mean(R_recur_MAM_19).reshape(-1,1)

In [59]:
Q_net_MAM_93.shape

(1, 1)

In [60]:
Q_net_MAM_93_19 = np.concatenate((Q_net_MAM_93, Q_net_MAM_94, Q_net_MAM_95, Q_net_MAM_96,
                                 Q_net_MAM_97, Q_net_MAM_98, Q_net_MAM_99, Q_net_MAM_00, Q_net_MAM_01,
                                 Q_net_MAM_02, Q_net_MAM_03, Q_net_MAM_04, Q_net_MAM_05, Q_net_MAM_06,
                                 Q_net_MAM_07, Q_net_MAM_08, Q_net_MAM_09, Q_net_MAM_10, Q_net_MAM_11,
                                 Q_net_MAM_12, Q_net_MAM_13, Q_net_MAM_14, Q_net_MAM_15, Q_net_MAM_16,
                                 Q_net_MAM_17, Q_net_MAM_18, Q_net_MAM_19), axis = 0)
Q_net_MAM_93_19.shape

(27, 1)

In [61]:
ZAdv_MAM_93_19 = np.concatenate((ZAdv_MAM_93, ZAdv_MAM_94, ZAdv_MAM_95, ZAdv_MAM_96,
                                 ZAdv_MAM_97, ZAdv_MAM_98, ZAdv_MAM_99, ZAdv_MAM_00, ZAdv_MAM_01,
                                 ZAdv_MAM_02, ZAdv_MAM_03, ZAdv_MAM_04, ZAdv_MAM_05, ZAdv_MAM_06,
                                 ZAdv_MAM_07, ZAdv_MAM_08, ZAdv_MAM_09, ZAdv_MAM_10, ZAdv_MAM_11,
                                 ZAdv_MAM_12, ZAdv_MAM_13, ZAdv_MAM_14, ZAdv_MAM_15, ZAdv_MAM_16,
                                 ZAdv_MAM_17, ZAdv_MAM_18, ZAdv_MAM_19), axis = 0)
ZAdv_MAM_93_19.shape

(27, 1)

In [62]:
MAdv_MAM_93_19 = np.concatenate((MAdv_MAM_93, MAdv_MAM_94, MAdv_MAM_95, MAdv_MAM_96,
                                 MAdv_MAM_97, MAdv_MAM_98, MAdv_MAM_99, MAdv_MAM_00, MAdv_MAM_01,
                                 MAdv_MAM_02, MAdv_MAM_03, MAdv_MAM_04, MAdv_MAM_05, MAdv_MAM_06,
                                 MAdv_MAM_07, MAdv_MAM_08, MAdv_MAM_09, MAdv_MAM_10, MAdv_MAM_11,
                                 MAdv_MAM_12, MAdv_MAM_13, MAdv_MAM_14, MAdv_MAM_15, MAdv_MAM_16,
                                 MAdv_MAM_17, MAdv_MAM_18, MAdv_MAM_19), axis = 0)
MAdv_MAM_93_19.shape

(27, 1)

In [63]:
VAdv_MAM_93_19 = np.concatenate((VAdv_MAM_93, VAdv_MAM_94, VAdv_MAM_95, VAdv_MAM_96,
                                 VAdv_MAM_97, VAdv_MAM_98, VAdv_MAM_99, VAdv_MAM_00, VAdv_MAM_01,
                                 VAdv_MAM_02, VAdv_MAM_03, VAdv_MAM_04, VAdv_MAM_05, VAdv_MAM_06,
                                 VAdv_MAM_07, VAdv_MAM_08, VAdv_MAM_09, VAdv_MAM_10, VAdv_MAM_11,
                                 VAdv_MAM_12, VAdv_MAM_13, VAdv_MAM_14, VAdv_MAM_15, VAdv_MAM_16,
                                 VAdv_MAM_17, VAdv_MAM_18, VAdv_MAM_19), axis = 0)
VAdv_MAM_93_19.shape

(27, 1)

In [64]:
R_origin_MAM_93_19  = np.concatenate((R_origin_MAM_93 , R_origin_MAM_94, R_origin_MAM_95, R_origin_MAM_96,
                                 R_origin_MAM_97, R_origin_MAM_98, R_origin_MAM_99, R_origin_MAM_00, R_origin_MAM_01,
                                 R_origin_MAM_02, R_origin_MAM_03, R_origin_MAM_04, R_origin_MAM_05, R_origin_MAM_06,
                                 R_origin_MAM_07, R_origin_MAM_08, R_origin_MAM_09, R_origin_MAM_10, R_origin_MAM_11,
                                 R_origin_MAM_12, R_origin_MAM_13, R_origin_MAM_14, R_origin_MAM_15, R_origin_MAM_16,
                                 R_origin_MAM_17, R_origin_MAM_18, R_origin_MAM_19), axis = 0)
R_origin_MAM_93_19.shape

(27, 1)

In [65]:
dT_dt_MAM_93_19  = np.concatenate((dT_dt_MAM_93 , dT_dt_MAM_94, dT_dt_MAM_95, dT_dt_MAM_96,
                                 dT_dt_MAM_97, dT_dt_MAM_98, dT_dt_MAM_99, dT_dt_MAM_00, dT_dt_MAM_01,
                                 dT_dt_MAM_02, dT_dt_MAM_03, dT_dt_MAM_04, dT_dt_MAM_05, dT_dt_MAM_06,
                                 dT_dt_MAM_07, dT_dt_MAM_08, dT_dt_MAM_09, dT_dt_MAM_10, dT_dt_MAM_11,
                                 dT_dt_MAM_12, dT_dt_MAM_13, dT_dt_MAM_14, dT_dt_MAM_15, dT_dt_MAM_16,
                                 dT_dt_MAM_17, dT_dt_MAM_18, dT_dt_MAM_19), axis = 0)
dT_dt_MAM_93_19.shape

(27, 1)

In [66]:
R_recur_MAM_93_19  = np.concatenate((R_recur_MAM_93 , R_recur_MAM_94, R_recur_MAM_95, R_recur_MAM_96,
                                 R_recur_MAM_97, R_recur_MAM_98, R_recur_MAM_99, R_recur_MAM_00, R_recur_MAM_01,
                                 R_recur_MAM_02, R_recur_MAM_03, R_recur_MAM_04, R_recur_MAM_05, R_recur_MAM_06,
                                 R_recur_MAM_07, R_recur_MAM_08, R_recur_MAM_09, R_recur_MAM_10, R_recur_MAM_11,
                                 R_recur_MAM_12, R_recur_MAM_13, R_recur_MAM_14, R_recur_MAM_15, R_recur_MAM_16,
                                 R_recur_MAM_17, R_recur_MAM_18, R_recur_MAM_19), axis = 0)
R_recur_MAM_93_19.shape

(27, 1)

In [67]:
np.savez(r'D:\heat_wave\atlantic\expand_area1\MLD_PLOT\MLD_equa_MAM.npz',
        Q_net_MAM_93_19 = Q_net_MAM_93_19, ZAdv_MAM_93_19 = ZAdv_MAM_93_19,
        MAdv_MAM_93_19 = MAdv_MAM_93_19, VAdv_MAM_93_19 = VAdv_MAM_93_19,
        R_origin_MAM_93_19 = R_origin_MAM_93_19, dT_dt_MAM_93_19 = dT_dt_MAM_93_19,
        R_recur_MAM_93_19 = R_recur_MAM_93_19)